In [ ]:
# !pip install -q deeppavlov
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import transformers
from transformers import DistilBertTokenizerFast, DistilBertForTokenClassification, AutoTokenizer, AutoModelForTokenClassification, pipeline
from sklearn.model_selection import train_test_split
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer

In [ ]:
# !python -m deeppavlov install ner_rus_convers_distilrubert_2L

In [ ]:
# from deeppavlov import build_model

# ner_model = build_model('ner_rus_convers_distilrubert_2L', download=True, install=True)

In [ ]:
# ner_model(['Сбер сделал выручку в 100 млн рублей'])

In [ ]:
ner_descrp = pd.read_csv('/kaggle/input/data-for-ner/mentions.csv')
ner_data = pd.read_pickle('/kaggle/input/data-for-ner/mentions texts.pickle')
ner_data

In [ ]:
ner_data = ner_data[ner_data['issuerid']>0]

In [ ]:
ner_data.shape

In [ ]:
ner_data['MessageText'][0]

In [ ]:
ner_data[ner_data['issuerid'] == 88]['MessageText'][355]

In [ ]:
tokenizer = WordPunctTokenizer()
stops = stopwords.words('russian') 
stop_marks = ["!", ">", "<", ".", ",", "@", "'", "#", "$", "%", "^", "&", "*", "(", ")", "-", '_', '=', '+', "№", ";", ":", "?", "`", "~", '/', '|', '[', ']', '{', '}']

In [ ]:
def tokenize_words(trash_data: list) -> pd.DataFrame:
    pass

In [ ]:
list_of_trash = ner_data['MessageText'].tolist()
list_of_trash[3]

In [ ]:
for_tokens_list = []
for sent in list_of_trash:
    for_tokens_list.append(re.sub(r'http\S+', '', sent))

for_tokens_list[0]

In [ ]:
tokenized_list_of_trash = [tokenizer.tokenize(sent.lower()) for sent in for_tokens_list]
tokenized_list_of_trash[0]

In [ ]:
def clean_text(sentences, stop_words, stop_marks):
    cleaned_sentences = []
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  
        u"\U0001F300-\U0001F5FF" 
        u"\U0001F680-\U0001F6FF"  
        u"\U0001F700-\U0001F77F" 
        u"\U0001F780-\U0001F7FF"  
        u"\U0001F800-\U0001F8FF" 
        u"\U0001F900-\U0001F9FF"  
        u"\U0001FA00-\U0001FA6F" 
        u"\U0001FA70-\U0001FAFF"  
        u"\U00002702-\U000027B0"  
        u"\U000024C2-\U0001F251"  
        "]",
        flags=re.UNICODE
    )

    for sent in sentences:
        helper = []
        for word in sent:
            word = emoji_pattern.sub('', word)
            cleaned_word = re.sub(r'[^\w\s]', '', word)
            if cleaned_word and cleaned_word not in stop_words and cleaned_word not in stop_marks:
                helper.append(cleaned_word)
        cleaned_sentences.append(helper)
    return cleaned_sentences

clean_sents = clean_text(tokenized_list_of_trash, stops, stop_marks)
clean_sents[0]

In [ ]:
sentences = []
seen = set() 

for sent in clean_sents:
    sentence = ' '.join(sent).strip()
    if sentence not in seen: 
        seen.add(sentence) 
        sentences.append(sentence) 

In [62]:
sentences[13]

'yndx vkco яндекс раскрыл сумму сделки продажи дзена vk 38 5 млн руб'

In [ ]:
tokenizer_bert = AutoTokenizer.from_pretrained("xlm-roberta-large-finetuned-conll03-english")
model = AutoModelForTokenClassification.from_pretrained("xlm-roberta-large-finetuned-conll03-english")
classifier = pipeline("ner", model=model, tokenizer=tokenizer_bert)

In [50]:
print(classifier(sentences[10]))

[{'entity': 'I-LOC', 'score': 0.9996332, 'index': 21, 'word': '▁с', 'start': 88, 'end': 89}, {'entity': 'I-LOC', 'score': 0.99947315, 'index': 22, 'word': 'ша', 'start': 89, 'end': 91}, {'entity': 'I-ORG', 'score': 0.9912714, 'index': 37, 'word': '▁ц', 'start': 164, 'end': 165}, {'entity': 'I-ORG', 'score': 0.99675155, 'index': 38, 'word': 'б', 'start': 165, 'end': 166}, {'entity': 'I-ORG', 'score': 0.9967026, 'index': 39, 'word': '▁', 'start': 167, 'end': 168}, {'entity': 'I-ORG', 'score': 0.9740354, 'index': 40, 'word': 'рф', 'start': 167, 'end': 169}, {'entity': 'I-ORG', 'score': 0.99860746, 'index': 43, 'word': '▁tes', 'start': 177, 'end': 180}, {'entity': 'I-ORG', 'score': 0.9981989, 'index': 44, 'word': 'la', 'start': 180, 'end': 182}, {'entity': 'I-ORG', 'score': 0.9986559, 'index': 45, 'word': '▁coca', 'start': 183, 'end': 187}, {'entity': 'I-ORG', 'score': 0.9987808, 'index': 46, 'word': '▁cola', 'start': 188, 'end': 192}, {'entity': 'I-ORG', 'score': 0.96431047, 'index': 52, 

Ниже код не работает! Он нужен для дообучения NER если я успею, а выше код на готовой моделе.


In [104]:
ner = pd.read_excel('/kaggle/input/new-fixed-data/names and synonyms.xlsx').drop(['issuerid', 'EMITENT_FULL_NAME'], axis=1)
ner

,VeryOddCompany,BGTicker,BGTicker.1,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,NaN,NaN,DERZP,Держава,DERZHAVA,DERZ,NaN,NaN,NaN,Держава,NaN,NaN,NaN
1,NaN,CBOM RX,NaN,Московский кредитный банк,мкб,Credit Bank of Moscow,Credit Bank,NaN,NaN,Московский кредитный банк,NaN,NaN,NaN
2,NaN,NaN,RDRB,Российский акционерный коммерческий дорожный банк,РДБанк,Дорожный банк,Russian public joint-stock commercial roads Bank,RosDorBank,РосДорБанк,Российский акционерный коммерческий дорожный банк,NaN,NaN,NaN
3,NaN,ALRS RX,NaN,алроса,alrosa,NaN,NaN,NaN,NaN,алроса,NaN,NaN,NaN
4,NaN,NaN,AVAN,Авангард,AVANGARD,NaN,NaN,NaN,NaN,Авангард,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,NaN,NaN,HNFG,Хендерсон,NaN,NaN,NaN,NaN,NaN,Хендерсон,NaN,NaN,NaN
251,NaN,NaN,SVCB,Sovcombank,NaN,NaN,NaN,NaN,NaN,Sovcombank,NaN,NaN,NaN
252,NaN,NaN,EUTR,АЗС Трасса,"АЗС ""Трасса""",АЗС «Трасса»,NaN,NaN,NaN,АЗС Трасса,NaN,NaN,NaN
253,NaN,NaN,DELI,Делимобил,Каршеринг Рус,delimobil,"ПАО ""Каршеринг Руссия""","""Каршеринг Руссия"", ПАО",NaN,Делимобил,NaN,NaN,NaN


In [119]:
fixed_ner_first = ner['Unnamed: 11']
all_new = ner[['Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11']]
ner = pd.concat([fixed_ner_first, all_new], axis=1)
ner

,Unnamed: 11,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,Держава,Держава,DERZHAVA,DERZ,NaN,NaN,NaN,Держава
1,Московский кредитный банк,Московский кредитный банк,мкб,Credit Bank of Moscow,Credit Bank,NaN,NaN,Московский кредитный банк
2,Российский акционерный коммерческий дорожный банк,Российский акционерный коммерческий дорожный банк,РДБанк,Дорожный банк,Russian public joint-stock commercial roads Bank,RosDorBank,РосДорБанк,Российский акционерный коммерческий дорожный банк
3,алроса,алроса,alrosa,NaN,NaN,NaN,NaN,алроса
4,Авангард,Авангард,AVANGARD,NaN,NaN,NaN,NaN,Авангард
...,...,...,...,...,...,...,...,...
250,Хендерсон,Хендерсон,NaN,NaN,NaN,NaN,NaN,Хендерсон
251,Sovcombank,Sovcombank,NaN,NaN,NaN,NaN,NaN,Sovcombank
252,АЗС Трасса,АЗС Трасса,"АЗС ""Трасса""",АЗС «Трасса»,NaN,NaN,NaN,АЗС Трасса
253,Делимобил,Делимобил,Каршеринг Рус,delimobil,"ПАО ""Каршеринг Руссия""","""Каршеринг Руссия"", ПАО",NaN,Делимобил


In [120]:
def remove_nans(values_list):
    return [value.lower() for value in values_list if not pd.isna(value)]

data = {}
for i in range(len(ner.values)):
    data[ner.values[i][0]] = list(set(ner.values[i]))

for key in data.keys():
    data[key] = remove_nans(data[key])
len(data.keys())

207

In [ ]:
entity_index = {}
for key in data:
    cleaned_list = remove_nans(data[key])
    for entity in cleaned_list:
        entity_index[entity.lower()] = 'ORG' 

def tag_sentence(sentence, entity_index):
    words = sentence.split()
    tags = []
    for word in words:
        if word.lower() in entity_index:
            tags.append('ORG')
        else:
            tags.append('O')
    return tags
tags = []
for i in range(len(sentences)):
    tags.append(tag_sentence(sentence, entity_index))
tags

In [155]:
new_data = {
    'Text': sentences,
    'Tags': tags
}

In [156]:
df = pd.DataFrame(new_data)

In [163]:
df.to_csv('tags_for_ner.csv')